# DNN

In [25]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

## 데이터 가져오기

In [26]:
train_set = pd.read_csv('train_set.csv')
train_set = train_set[train_set['ECLO'] <= 25]
test_set = pd.read_csv('test_set.csv')

## train test valid 분리

In [40]:
train_x, test_x, train_y, test_y = train_test_split(train_set[['요일', '사고시간', '기상상태', '도로형태', '노면상태', '사고유형']], train_set['ECLO'], test_size=0.2)

In [41]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2)

## tensorflow model 정의

In [42]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(6)),
    tf.keras.layers.Embedding(input_dim=6, output_dim=100),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.LSTM(16, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1)]
)

In [43]:
checkpoint_filepath = "logs/LSTM/"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
)

In [44]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredLogarithmicError(), metrics=['mae'])
model.fit(train_x, train_y, epochs=100, validation_data=(valid_x, valid_y),
          callbacks=[
                        checkpoint_callback,
                        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3),
                    ])

Epoch 1/100
791/791 [==============================] - 18s 17ms/step - loss: 0.2664 - mae: 2.1395 - val_loss: 0.2036 - val_mae: 2.0839
Epoch 2/100
791/791 [==============================] - 12s 15ms/step - loss: 0.2017 - mae: 2.0507 - val_loss: 0.1994 - val_mae: 2.0593
Epoch 3/100
791/791 [==============================] - 11s 14ms/step - loss: 0.2009 - mae: 2.0497 - val_loss: 0.2035 - val_mae: 2.0840
Epoch 4/100
791/791 [==============================] - 12s 15ms/step - loss: 0.2008 - mae: 2.0509 - val_loss: 0.2001 - val_mae: 2.0647
Epoch 5/100
791/791 [==============================] - 11s 14ms/step - loss: 0.2008 - mae: 2.0502 - val_loss: 0.1981 - val_mae: 2.0279
Epoch 6/100
791/791 [==============================] - 11s 14ms/step - loss: 0.2007 - mae: 2.0501 - val_loss: 0.1988 - val_mae: 2.0528
Epoch 7/100
791/791 [==============================] - 11s 14ms/step - loss: 0.2004 - mae: 2.0498 - val_loss: 0.1982 - val_mae: 2.0430
Epoch 8/100
791/791 [==============================] - 

In [45]:
np.sqrt(mean_squared_log_error(test_y, model.predict(test_x)))

248/248 [==============================] - 2s 5ms/step


0.448665095926236

## 결과 저장

In [46]:
ECLOs = model.predict(test_set[['요일', '사고시간', '기상상태', '도로형태', '노면상태', '사고유형']])
ans = pd.read_csv('open/sample_submission.csv')
ans['ECLO'] = ECLOs
ans.to_csv('ans_lstm.csv', index=False)

343/343 [==============================] - 2s 5ms/step
